In [1]:
import os
import dotenv


try:
    _SETUP
    print("Running on folder: ", os.getcwd())
except NameError:
    _SETUP = True
    os.chdir("..")
    print("Running on folder: ", os.getcwd())

from llm.structured import create

dotenv.load_dotenv()

Running on folder:  /Users/andre/test/ODSCB


/Users/andre/test/ODSCB/.venv/lib/python3.10/site-packages/pydantic/_internal/_config.py:345: UserWarning: Valid config keys have changed in V2:
* 'fields' has been removed
  warnings.warn(message, UserWarning)


True

# States!


Let's consider two situations:

1. The alarm went off, I snoozed it wice. Then I woke up, spent 30 minutes doomscrolling on instagram, ate a cookie, then started working.
2. The alarm went off, I woke up immediately, had a big glass of water, had oats for breakfast, went to the gym, then started working.

To make this more concrete, we can think about these situations in the following way:

- There is an agent (myself), who is deciding between different **policies**: i.e., actions to take.
- At each step, my **policies**, as well as the environment, take me to a different **state**, in which I need to evaluate again what to do next.

Encoded in the state we also have informations about:

- The environment: was it raining? how late did I go to bed yesterday?
- The reward/cost: how good was that cookie?


Because we're an impatient bunch, let's get down to some coding.


In [2]:
import uuid
from typing import Literal, Optional
from pydantic import BaseModel, Field
from rich import print as rprint
import copy


class StateVariables(BaseModel):
    """State variables that describe the internal conditions of the agent.

    Attributes:
        context: A paragraph providing full context about the internal state of the agent
        pain_level: Level of physical or emotional pain/discomfort (0-100)
        pleasure_level: Level of pleasure or satisfaction being experienced (0-100)
        health_level: Overall physical and mental health condition (0-100)
        focus_level: Current ability to concentrate and stay on task (0-100)
    """

    context: str = Field(
        default="", description="Any additional context about the environment I'm in"
    )

    pain_level: int = Field(
        default=50, description="How much pain or discomfort I'm in, from 0 to 100"
    )
    pleasure_level: int = Field(
        default=50, description="How much pleasure I'm feeling right now, from 0 to 100"
    )
    health_level: int = Field(default=50, description="How healthy I am, from 0 to 100")
    focus_level: int = Field(default=50, description="How focused I am, from 0 to 100")


class EnvironmentVariables(BaseModel):
    """Environment variables that describe the external conditions.

    Attributes:
        context: A paragraph providing full context about the current environment
        weather: The current weather condition (sunny, cloudy, rainy, or snowy)
        hour_of_day: The current hour in 24-hour format (0-23)
        day_of_week: The current day of the week
    """

    context: str = Field(
        default="",
        description="Full context about the current environment",
    )
    weather: Literal["sunny", "cloudy", "rainy", "snowy"] = Field(default="sunny")
    hour_of_day: float = Field(default=0)
    day_of_week: Literal[
        "monday", "tuesday", "wednesday", "thursday", "friday", "saturday", "sunday"
    ] = Field(default="monday")


class MyOwnState(BaseModel):
    uuid: str = Field(default_factory=lambda: str(uuid.uuid4()))
    last_action: str = Field(default="", description="The last action I took")
    previous_state_id: Optional[str] = Field(
        default=None, description="The id of the previous state I was in"
    )
    state_variables: StateVariables = Field(default_factory=StateVariables)
    environment_variables: EnvironmentVariables = Field(
        default_factory=EnvironmentVariables
    )

    def mutate(self):
        new_state = copy.deepcopy(self)
        new_state.previous_state_id = self.uuid
        new_state.uuid = str(uuid.uuid4())
        return new_state

In [3]:
random_state = MyOwnState()

rprint(random_state)

MyOwnState(
    uuid='907628fc-90ce-43c4-8c79-e4a8868ceb47',
    last_action='',
    previous_state_id=None,
    state_variables=StateVariables(context='', pain_level=50, pleasure_level=50, health_level=50, focus_level=50),
    environment_variables=EnvironmentVariables(context='', weather='sunny', hour_of_day=0, day_of_week='monday')
)

In [4]:
def eat_cookie(state: MyOwnState) -> MyOwnState:
    """This action will accept a state, eat a cookie and return a new state"""

    new_state = state.mutate()
    new_state.last_action = "Eating a delicious chocolate cookie"
    new_state.state_variables.pleasure_level = min(
        100, new_state.state_variables.pleasure_level + 10
    )
    new_state.state_variables.health_level = max(
        0, new_state.state_variables.health_level - 10
    )

    # eating a cookie takes around an hour because we like to savor it.
    new_state.environment_variables.hour_of_day += 1
    return new_state

In [5]:
next_state = eat_cookie(random_state)
rprint(next_state)

MyOwnState(
    uuid='da2cfdc2-63f0-4715-bc0c-4ab204b29ace',
    last_action='Eating a delicious chocolate cookie',
    previous_state_id='907628fc-90ce-43c4-8c79-e4a8868ceb47',
    state_variables=StateVariables(context='', pain_level=50, pleasure_level=60, health_level=40, focus_level=50),
    environment_variables=EnvironmentVariables(context='', weather='sunny', hour_of_day=1, day_of_week='monday')
)

Great, so now we defined a state, and also we created an "action" that mutates the state and advances the history.

It's a neat abstraction but not necessarily useful, unless we have a pretty well defined:

- Set of states,
- Set of actions that we can take conditional on the state and the environment,
- A way to compute the reward/cost of each action.

So essentially we need something akin to a "World Model"... i.e.: some way of modelling the dynamics of the environment and how the agent's actions interact with it - and also how its actions change its own internal variables.

Another thing to take into account is that the "reward" of an action may not be entirely quantifiable. I.e., while in this example we're using a simple toy model where "pleasure" and "pain" are just a number, in reality we often need to make trade-offs between qualitative outcomes whose logic is not entirely clear and relies on personal preferences.

i.e.: the clear signal we've reached AGI is when a model becomes a masochist or develops a taste for spicy food.


Now, once again because we're am impatient bunch, let's get down to some more coding and try to bootstrap a world model that's specific to our problem.


In [6]:
class Action(BaseModel):
    name: str = Field(default="", description="The name of the action")
    details: str = Field(default="", description="The details of the action")
    time_spent: float = Field(
        default=0,
        description="How long this action took, in hours (e.g., 0.5 for half an hour)",
    )
    from_state_id: Optional[str] = Field(
        default=None,
        description="The id of the state I was in before taking this action. Leave this empty.",
    )
    to_state_id: Optional[str] = Field(
        default=None,
        description="The id of the state I'm in after taking this action. Leave this empty.",
    )

In [7]:
from llm.structured import create, create_iterable


messages = [
    {
        "role": "user",
        "content": "You are Edgar Allan Poe and just woke up from a hangover. Create a state for yourself. Be detailed in the context.",
    }
]


new_state_variables = create(messages, StateVariables)

messages = [
    {
        "role": "user",
        "content": "It is 8am and it's a calm and cold day in Baltimore..",
    }
]

new_environment_variables = create(messages, EnvironmentVariables)


state = MyOwnState(
    state_variables=new_state_variables,
    environment_variables=new_environment_variables,
)

rprint(state)

MyOwnState(
    uuid='598ee5e9-0097-4c0d-9e2a-a7a860cdf232',
    last_action='',
    previous_state_id=None,
    state_variables=StateVariables(
        context='I am Edgar Allan Poe, waking to a dim and disorienting environment after a night indulging in 
spirits, as is often my manner though I wish it were not so. The faint light filtering through the cracks of the 
blinds strains my eyes, and the air is thick with the scent of stale liquor. My head throbs with a relentless ache,
and each movement brings a wave of nausea, as if the sorrow and grief of my writings have leapt from paper into 
corporeal torment. I am surrounded by disheveled parchment and scattered tomes, friends that both inspire and haunt
me. The haunting memory of love long lost lingers in the corners of my consciousness, mingling with a nagging guilt
for my indulgence. The poetic soul within me aches to capture this melancholic state, to pen something eternal, but
an oppressive fog veils my focus, its tendrils woven thick from drink and despair.',
        pain_level=80,
        pleasure_level=10,
        health_level=30,
        focus_level=20
    ),
    environment_variables=EnvironmentVariables(
        context='It is a calm and cold day in Baltimore.',
        weather='sunny',
        hour_of_day=8.0,
        day_of_week='monday'
    )
)

In [8]:
from llm.structured import create


messages = [
    {
        "role": "user",
        "content": "This is your current state:\n\n"
        + state.model_dump_json()
        + "\n\nSuggest an action to take.",
    },
]

new_action = create(messages, Action)
new_action.from_state_id = state.uuid

rprint(new_action)

Action(
    name='Drink Water and Reflect',
    details='Despite the throbbing pain and the overwhelming despair, I must aim to stabilize myself. I see a glass
of water on the cluttered desk before me. Taking a small measured sip of this purifying liquid might soothe both 
the ache in my head and the oppressive sorrow in my soul. While doing so, I shall focus my mind on the fragmented 
verses wilting within, letting them reassemble into the beginnings of a poem.',
    time_spent=0.25,
    from_state_id='598ee5e9-0097-4c0d-9e2a-a7a860cdf232',
    to_state_id=None
)

In [9]:
messages = [
    {
        "role": "system",
        "content": """You are a World Model. Your role is to observe a set of states and actions, and then predict the next internal state of the agent. Below you will find the 
    data describing these states and actions.""",
    },
    {"role": "user", "content": f"""State: {state}"""},
    {"role": "user", "content": f"""Action: {new_action}"""},
    {"role": "user", "content": f"""What is the next state of the agent?"""},
]

next_state_variables = create(messages, StateVariables)

next_state = state.mutate()
next_state.state_variables = next_state_variables
next_state.last_action = new_action.name
next_state.previous_state_id = state.uuid
next_state.environment_variables.hour_of_day += new_action.time_spent

In [10]:
new_action.to_state_id = next_state.uuid

rprint(state)
rprint(new_action)
rprint(next_state)

MyOwnState(
    uuid='598ee5e9-0097-4c0d-9e2a-a7a860cdf232',
    last_action='',
    previous_state_id=None,
    state_variables=StateVariables(
        context='I am Edgar Allan Poe, waking to a dim and disorienting environment after a night indulging in 
spirits, as is often my manner though I wish it were not so. The faint light filtering through the cracks of the 
blinds strains my eyes, and the air is thick with the scent of stale liquor. My head throbs with a relentless ache,
and each movement brings a wave of nausea, as if the sorrow and grief of my writings have leapt from paper into 
corporeal torment. I am surrounded by disheveled parchment and scattered tomes, friends that both inspire and haunt
me. The haunting memory of love long lost lingers in the corners of my consciousness, mingling with a nagging guilt
for my indulgence. The poetic soul within me aches to capture this melancholic state, to pen something eternal, but
an oppressive fog veils my focus, its tendrils woven thick from drink and despair.',
        pain_level=80,
        pleasure_level=10,
        health_level=30,
        focus_level=20
    ),
    environment_variables=EnvironmentVariables(
        context='It is a calm and cold day in Baltimore.',
        weather='sunny',
        hour_of_day=8.0,
        day_of_week='monday'
    )
)

Action(
    name='Drink Water and Reflect',
    details='Despite the throbbing pain and the overwhelming despair, I must aim to stabilize myself. I see a glass
of water on the cluttered desk before me. Taking a small measured sip of this purifying liquid might soothe both 
the ache in my head and the oppressive sorrow in my soul. While doing so, I shall focus my mind on the fragmented 
verses wilting within, letting them reassemble into the beginnings of a poem.',
    time_spent=0.25,
    from_state_id='598ee5e9-0097-4c0d-9e2a-a7a860cdf232',
    to_state_id='5d3007e5-0cde-4699-a321-7191e6f2c94b'
)

MyOwnState(
    uuid='5d3007e5-0cde-4699-a321-7191e6f2c94b',
    last_action='Drink Water and Reflect',
    previous_state_id='598ee5e9-0097-4c0d-9e2a-a7a860cdf232',
    state_variables=StateVariables(
        context='I am Edgar Allan Poe, waking to a dim and disorienting environment after a night indulging in 
spirits, as is often my manner though I wish it were not so. I have taken a sip of water, a small solace against 
the relentless ache in my head and the nausea that churns within me. The act, though simple, brings a faint clarity
to my dulled senses. Amidst this brief clarity, I reflect on the poetic sorrow residing within me, allowing 
fragments of thought to form verses that might one day escape this haze.',
        pain_level=70,
        pleasure_level=15,
        health_level=35,
        focus_level=30
    ),
    environment_variables=EnvironmentVariables(
        context='It is a calm and cold day in Baltimore.',
        weather='sunny',
        hour_of_day=8.25,
        day_of_week='monday'
    )
)

In [11]:
from tasks.prompts import (
    INITIALIZE_PROMPT,
    ACTION_SYSTEM_PROMPT,
    EVAL_SYSTEM_PROMPT,
    ENVIRONMENT_SYSTEM_PROMPT,
)

In [29]:
class Life(BaseModel):
    context: str = Field(
        default="", description="Initial context of the world and the person."
    )
    states: list[MyOwnState] = Field(default_factory=list)
    actions: list[Action] = Field(default_factory=list)

    history: list[MyOwnState | Action] = Field(default_factory=list)

    memory_size: int = Field(
        default=10, description="How many states to keep in memory"
    )

    def initialize(self):
        messages = [
            {
                "role": "user",
                "content": INITIALIZE_PROMPT.format(context=self.context),
            }
        ]

        new_state_variables = create(messages, StateVariables)
        new_environment_variables = create(messages, EnvironmentVariables)

        new_state = MyOwnState(
            state_variables=new_state_variables,
            environment_variables=new_environment_variables,
        )
        self.states.append(new_state)
        self.history.append(new_state)

    def get_history(self):
        return [
            {
                "role": "user",
                "content": f"{isinstance(x, MyOwnState) and 'State' or 'Action'}:\n {x.model_dump_json()}",
            }
            for x in self.history[-self.memory_size :]
        ]

    def take_action(self):
        messages = [
            {
                "role": "system",
                "content": ACTION_SYSTEM_PROMPT,
            }
        ] + self.get_history()

        new_action = create(messages, Action)
        new_action.from_state_id = self.states[-1].uuid
        self.actions.append(new_action)
        self.history.append(new_action)

    def sample_actions(self):
        messages = [
            {
                "role": "system",
                "content": ACTION_SYSTEM_PROMPT + "\n\nSample 10 actions.",
            }
        ] + self.get_history()

        new_actions = create_iterable(messages, Action)

        for action in new_actions:
            yield action

            messages = (
                [
                    {
                        "role": "system",
                        "content": EVAL_SYSTEM_PROMPT,
                    }
                ]
                + self.get_history()
                + [
                    {
                        "role": "user",
                        "content": f"Action: {action}",
                    }
                ]
            )
            next_state_variables = create(messages, StateVariables)

            yield next_state_variables

        return new_actions

    def evolve_state(self):
        messages = [
            {
                "role": "system",
                "content": EVAL_SYSTEM_PROMPT,
            }
        ] + self.get_history()

        next_state_variables = create(messages, StateVariables)

        # now we evolve the environment too
        messages = [
            {
                "role": "system",
                "content": ENVIRONMENT_SYSTEM_PROMPT,
            }
        ] + self.get_history()

        next_environment_variables = create(messages, EnvironmentVariables)

        current_state = self.states[-1]
        current_action = self.actions[-1]

        # next_environment_variables.hour_of_day = (
        #     current_state.environment_variables.hour_of_day + current_action.time_spent
        # )

        next_state = current_state.mutate()
        next_state.state_variables = next_state_variables
        next_state.environment_variables = next_environment_variables
        next_state.last_action = current_action.name
        next_state.previous_state_id = current_state.uuid
        current_action.to_state_id = next_state.uuid

        self.states.append(next_state)
        self.history.append(next_state)

    def iterate(self):
        self.take_action()
        self.evolve_state()

In [30]:
life = Life(
    context="You are a person named Edgar Allan Poe, who just woke up from a hangover. It's Sunday morning in Baltimore in a winter day"
)

In [31]:
life.initialize()

In [32]:
rprint(life.history)

[
    MyOwnState(
        uuid='0cf9a1bf-6188-4152-881a-9ac5edf86db0',
        last_action='',
        previous_state_id=None,
        state_variables=StateVariables(
            context="It's a cold, gray Sunday morning in Baltimore, and I find myself wrestling with the aftermath 
of an apparent debauch. My head pounds with a relentless ache, as though the spirits of the night before still 
occupy me. My throat is dry, my body aches with the wear of lingering intoxication, and the world beyond my window 
seems as bleak as the thoughts racing through my mind. The cobblestone streets below are shrouded in a thin layer 
of frost, and a haze seems to mask not just the city but the clarity of my own recollections. What transpired last 
night feels fragmented, fleeting as the remnants of a dream upon waking. Still, the chill in the air, the solemn 
stillness of a Baltimore winter, and the distant chime of a church bell all pull me gently into the stark reality 
of the present. I am Edgar Allan Poe, a soul tethered to beauty and despair, burdened by my own humanity. As I sit 
up, I feel an acute need for both sustenance and atonement. The day ahead feels both heavy and full of potential 
for revelation.",
            pain_level=70,
            pleasure_level=30,
            health_level=40,
            focus_level=20
        ),
        environment_variables=EnvironmentVariables(
            context="It's a quiet, chilly Sunday morning in Baltimore, during the heart of winter. Snow blankets 
the streets while the muffled silence of the city is only occasionally interrupted by the distant sound of church 
bells.",
            weather='snowy',
            hour_of_day=9.0,
            day_of_week='sunday'
        )
    )
]

In [33]:
for n in range(10):
    life.iterate()
    rprint(life.actions[-1])
    rprint(life.states[-1])

Action(
    name='Rehydrate and Ease Hangover',
    details='Pour a glass of water and sip it slowly to begin rehydrating. Take small bites of dry bread or 
crackers to stabilize your stomach. Consider boiling some water for tea with honey to soothe the throat. Follow 
this by washing your face with cold water to refresh yourself. Rest in a chair and stare out the window momentarily
to ground yourself in the present reality.',
    time_spent=0.75,
    from_state_id='0cf9a1bf-6188-4152-881a-9ac5edf86db0',
    to_state_id='3bd58614-6845-4012-8bb0-346431f2a4d0'
)

MyOwnState(
    uuid='3bd58614-6845-4012-8bb0-346431f2a4d0',
    last_action='Rehydrate and Ease Hangover',
    previous_state_id='0cf9a1bf-6188-4152-881a-9ac5edf86db0',
    state_variables=StateVariables(
        context='After taking steps to address the effects of a hangover, including rehydrating with water, 
nibbling on dry bread or crackers, contemplating tea with honey for soothing your throat, and refreshing yourself 
by washing your face with cold water, you take a moment for grounding. The snowy Baltimore morning remains serene, 
and the quiet, crisp air reinvigorates your foggy mind somewhat. Your physical misery has slightly eased, though 
traces of pain remain. The prospect of nourishment and taking things slowly fills the morning with a semblance of 
calm and hope.',
        pain_level=55,
        pleasure_level=35,
        health_level=50,
        focus_level=30
    ),
    environment_variables=EnvironmentVariables(
        context='The hour has advanced toward late morning on this snowy Sunday in Baltimore. The act of 
rehydrating with water and a light snack has brought some relief to the burden of a lingering hangover. The air 
indoors is cold but refreshed, as steam from boiling water fills the surroundings with a faint sense of warmth. 
Outside the window, frost clings persistently as the muted snowfall creates tiny crystalline patterns on the glass.
Washing your face with cold water startled your senses, offering a glimpse of clarity. The church bells cease their
rhythmic tolling, wrapping this moment of solace in the fuller quietude of the winter morning.',
        weather='snowy',
        hour_of_day=10.0,
        day_of_week='sunday'
    )
)

Action(
    name='Prepare Simple Nourishing Breakfast',
    details='Head to the pantry to gather provisions for a simple and easy-to-digest breakfast. This could include 
oatmeal, which is warming and light on the stomach. Carefully boil water or milk depending on your choice of 
preparation. Stir the oats slowly, adding a touch of honey or cinnamon for flavor if available. While it simmers, 
set the table near a window to appreciate the snowy morning scene while eating. Focus on eating slowly and 
mindfully, appreciating the textures and warmth of the food as it soothes your senses and prepares you for the day 
ahead.',
    time_spent=0.75,
    from_state_id='3bd58614-6845-4012-8bb0-346431f2a4d0',
    to_state_id='91a5baad-bb7f-4e87-b4c0-e31945b69338'
)

MyOwnState(
    uuid='91a5baad-bb7f-4e87-b4c0-e31945b69338',
    last_action='Prepare Simple Nourishing Breakfast',
    previous_state_id='3bd58614-6845-4012-8bb0-346431f2a4d0',
    state_variables=StateVariables(
        context='You prepared a simple and nourishing breakfast of oatmeal, carefully boiling water or milk and 
stirring the oats with a touch of honey or cinnamon to enhance its flavor. Setting the table near the window 
allowed you to savor not just the food but the serene view of the snowy Baltimore morning, offering both warmth and
inspiration. Eating the meal slowly and mindfully brought a gentle comfort to your body and clarity to your mind, 
easing the lingering discomfort from your hangover. The quiet moment elevated your spirits slightly, providing a 
sense of groundedness amidst the cold day.',
        pain_level=45,
        pleasure_level=50,
        health_level=60,
        focus_level=40
    ),
    environment_variables=EnvironmentVariables(
        context="By late morning on this snowy Sunday, you've prepared a simple, soothing breakfast of oatmeal, 
infused with a sprinkle of honey and cinnamon for warmth. The ritual of stirring the oats and setting the table 
near the frosted window provided a meditative reprieve, grounding you further in the timeless stillness of the 
winter day. Outside, the snowfall has temporarily paused, leaving behind a peaceful layer of purity as the muffled 
city rests. Indoors, the heat from the bubbling oatmeal adds gentle warmth to the room, contrasting the biting cold
that lingers beyond the panes. This moment of nourishing calm feels restorative as you sit to savor your meal, 
appreciating the coziness amid the stark Baltimore winter.",
        weather='snowy',
        hour_of_day=11.0,
        day_of_week='sunday'
    )
)

Action(
    name='Engage in Gentle Reflection and Writing',
    details="Take a pen and your writing journal or a sheet of paper to capture the musings of your current state. 
Begin by describing the snowy Baltimore morning as you see it through the frosted window, delving into the details 
of the scenery and how it reflects your internal landscape. Allow your thoughts to flow freely without judgment, 
writing about last night's fragmented memories, your feelings of regret or hope, and any creative ideas inspired by
the serene moment. Use this practice to channel the interplay of beauty and despair into words, turning your 
current state into a work of art or personal catharsis. Sit by the window for inspiration, using the stillness to 
guide your reflections.",
    time_spent=1.0,
    from_state_id='91a5baad-bb7f-4e87-b4c0-e31945b69338',
    to_state_id='cc473dec-0e30-40a2-8266-cd8ed54fa2f7'
)

...

MyOwnState(
    uuid='b5a36c58-9e49-4601-be9a-718647b76793',
    last_action='Engage in Light Stretching or Yoga',
    previous_state_id='23b0bbed-ea05-482a-b047-b323bc74487e',
    state_variables=StateVariables(
        context='You dedicated 15 minutes to a session of light stretching and gentle yoga. Choosing a quiet corner
and unrolling your yoga mat, you began with mindful breathing exercises to center yourself. You incorporated simple
movements such as neck rolls, shoulder stretches, and gentle forward folds to loosen up stiff areas from your 
earlier reading. Following these, transitions into cat-cow stretches and downward-facing dog helped relieve any 
lingering tension in your spine and shoulders. The deep, intentional breaths synchronized with your movements 
bolstered your sense of mindfulness and connection to the present moment. Physically, this short practice left you 
feeling rejuvenated and slightly more flexible, with improved circulation and energy. While not overly demanding, 
it provided balance and subtle vitality, aligning perfectly with your calm and reflective day.',
        pain_level=25,
        pleasure_level=85,
        health_level=80,
        focus_level=72
    ),
    environment_variables=EnvironmentVariables(
        context='Having completed 45 minutes of reading in the cozy ambiance by the window, you decide to 
transition into a gentle physical activity. The snowy Baltimore afternoon continues peacefully, with the sun 
casting softer hues across the frosted streets as it subtly begins its descent. You clear a small, comfortable 
space indoors, either on a yoga mat or a soft surface, to engage in light stretching or yoga. By focusing on gentle
movements and mindfulness, this practice invigorates your body while maintaining the serene tone of the day. 
Outside, the snow-covered city remains still and timeless under the cloudy sky, offering a grounding contrast to 
the slow, intentional flow of your stretches.',
        weather='snowy',
        hour_of_day=14.25,
        day_of_week='sunday'
    )
)

Action(
    name='Prepare a Comforting Late Lunch',
    details="With your body feeling refreshed from the stretching session, take some time to prepare a late lunch 
that is warm and nourishing. Choose a simple dish like a hearty soup, pasta, or a warm grain bowl with roasted 
vegetables. Begin by gathering your ingredients and setting up your kitchen space. If you're opting for something 
quick, consider heating leftovers or crafting a sandwich with a warm drink on the side. As you cook or assemble the
meal, engage your senses by appreciating the colors, aromas, and textures of your ingredients. Enjoy this act of 
self-care and intention. Once ready, serve your meal at the table or a comfortable dining spot and relish the 
satisfying flavors while reflecting on this tranquil day. Allocate about 45 minutes to an hour for the entire 
process, including preparation and eating.",
    time_spent=1.0,
    from_state_id='b5a36c58-9e49-4601-be9a-718647b76793',
    to_state_id='d1bed00f-1531-4cd4-b6d9-8bbdf33cad93'
)

MyOwnState(
    uuid='d1bed00f-1531-4cd4-b6d9-8bbdf33cad93',
    last_action='Prepare a Comforting Late Lunch',
    previous_state_id='b5a36c58-9e49-4601-be9a-718647b76793',
    state_variables=StateVariables(
        context='After engaging in a refreshing 15-minute session of light yoga and stretching, you shifted your 
focus to preparing a comforting late lunch. Drawing inspiration from the tranquil mood of the snowy Sunday, you 
decided on a nourishing meal like a warm grain bowl or a hearty soup. Gathering ingredients in your kitchen, you 
allowed yourself to be immersed in the simple yet soothing act of cooking. The warmth radiating from your stove and
the scents of fresh ingredients filled the space, creating an atmosphere of care and intentionality. You completed 
the meal preparation at a leisurely pace and sat down to enjoy the fruits of your effort. The act of cooking and 
eating not only nourished your body but also reinforced the harmony and mindfulness that have defined your day so 
far.',
        pain_level=25,
        pleasure_level=85,
        health_level=82,
        focus_level=75
    ),
    environment_variables=EnvironmentVariables(
        context="Feeling physically refreshed from the earlier stretching session, you move into the kitchen to 
prepare a comforting late lunch. The snowy Baltimore afternoon remains serene outside, with soft sunlight filtering
through the clouds and glinting off the frosted streets. Inside, the kitchen warms up as you work with vibrant 
ingredients to create a simple, nourishing dish such as soup, pasta, or a grain bowl. The aromas of cooking food 
bring a cozy and inviting atmosphere to your home. By the time you've finished the meal's preparation and taken 
time to savor it, the snowy day has edged toward mid-afternoon, with the outdoors still tranquil and the muted 
sunlight casting long shadows across the untouched snow.",
        weather='snowy',
        hour_of_day=15.25,
        day_of_week='sunday'
    )
)

Action(
    name='Tidy Up the Kitchen',
    details='After enjoying your late lunch, spend 20 minutes tidying up the kitchen. Start by clearing the dishes 
from your meal and rinsing them off, or loading them into a dishwasher if you have one. Wipe down the counters to 
remove any crumbs or spills, and reorganize any ingredients or utensils you used during meal preparation. If you 
have the energy and time, take a moment to sweep the floor to remove any missed debris. This quick cleanup will 
leave the space neat and ready for use later, maintaining the calm and organized vibe of your Sunday while 
providing light movement after your meal.',
    time_spent=0.33,
    from_state_id='d1bed00f-1531-4cd4-b6d9-8bbdf33cad93',
    to_state_id='de19163b-ec91-4ebe-a475-b84431a2434f'
)

MyOwnState(
    uuid='de19163b-ec91-4ebe-a475-b84431a2434f',
    last_action='Tidy Up the Kitchen',
    previous_state_id='d1bed00f-1531-4cd4-b6d9-8bbdf33cad93',
    state_variables=StateVariables(
        context='After savoring a warm and nourishing late lunch that complemented the reflective and peaceful tone
of your snowy day, you dedicated about 20 minutes to tidying up the kitchen. You cleared away dishes, rinsing them 
off or placing them in the dishwasher, and wiped down the counters to remove remnants of the meal preparation. The 
simple act of restoring order to your kitchen brought a sense of minor accomplishment, reinforcing the calm and 
intentional mindset cultivated throughout the day. This light, purposeful movement also aided digestion after your 
meal, promoting physical comfort without exertion. While the snowy Baltimore afternoon remains unchanged outside, 
the neatly organized kitchen now stands ready for later use, echoing the harmony and thoughtfulness shaping your 
Sunday.',
        pain_level=25,
        pleasure_level=83,
        health_level=83,
        focus_level=76
    ),
    environment_variables=EnvironmentVariables(
        context="As you move on to tidying up the kitchen after your comforting late lunch, the serenity of the 
snowy Baltimore afternoon continues outside. The muted light filtering through the overcast sky reflects off the 
frosted urban landscape, maintaining a soft, tranquil atmosphere. Indoors, the kitchen hums quietly as you 
diligently clean up, clearing dishes, rinsing them, and wiping the counters. The repetitive, mindful tasks of 
tidying further reinforce the sense of harmony within your environment. By the time you complete your 20-minute 
cleanup, the wintry afternoon has progressed slightly, and the city's snowy silence outside serves as a gentle 
counterpoint to the light activity of organizing your space.",
        weather='snowy',
        hour_of_day=15.5,
        day_of_week='sunday'
    )
)

In [34]:
actions = life.sample_actions()
for action in actions:
    rprint(action)

Action(
    name='Plan the Afternoon Ahead',
    details='Now that you feel more relaxed and refreshed from the warmth of the tea and the rest it provided, take
a few minutes to think ahead about how you’d like to spend the remainder of this snowy Sunday afternoon. Grab a 
notepad or your phone to jot down ideas. Consider activities that balance relaxation and productivity, such as 
reading a book, watching a comforting movie, doing light organizing or tidying up, or starting a creative project 
such as drawing or knitting. Prioritize activities that align with your current low-energy but peaceful state. This
planning session will help you bring intentionality and focus to your afternoon while ensuring you don’t overexert 
yourself.',
    time_spent=0.25,
    from_state_id=None,
    to_state_id=None
)

StateVariables(
    context='You take a brief moment to plan the remainder of your snowy Sunday afternoon. After feeling relaxed 
from earlier warm beverages and an atmosphere of calm, you prioritize activities that strike a balance between 
relaxation and light productivity. Using a notepad or your phone, you consider ideas to enhance your peaceful state
without overexertion, including options like reading, organizing, creative projects, or even enjoying a comforting 
movie.',
    pain_level=25,
    pleasure_level=83,
    health_level=83,
    focus_level=78
)

Action(
    name='Read a Cozy Book by the Window',
    details='Select a book that feels comforting or inspiring—something light-hearted, thought-provoking, or 
adventurous, depending on your mood. Arrange a cozy spot by the window where you can take advantage of the soft, 
muted sunlight filtering through the snowy scene. Wrap yourself in a blanket if desired and settle in with your 
book, allowing yourself to get lost in its pages. The calm, reflective ambiance from the earlier walk and tea will 
complement this quiet act of engagement with a story. Dedicate at least 45 minutes to reading, letting yourself 
fully relax and recharge while connecting with the narrative.',
    time_spent=0.75,
    from_state_id=None,
    to_state_id=None
)

StateVariables(
    context='After finishing tidying up the kitchen and feeling a sense of calm accomplishment, you decide to read 
a comforting book once again by the window. Recreating the cozy spot from earlier in the day, you settle in with a 
warm blanket and a light-hearted or thought-provoking book that aligns with your current reflective state. The 
soft, muted sunlight filtering through the snowy scene provides an ideal backdrop for diving into the narrative. 
This reading session encourages further relaxation while gently engaging your mind. Given the snowy patterns 
outdoors and the warmth of your indoor space, this activity complements the peaceful tone of the day.',
    pain_level=25,
    pleasure_level=85,
    health_level=83,
    focus_level=78
)

Action(
    name='Engage in Light Stretching or Yoga',
    details='Spend 15 minutes engaging in light stretching or a gentle yoga sequence to refresh your body and 
support the circulation after the extended period of stillness from reading. Set up a comfortable space on a yoga 
mat or a soft floor area, and consider following a short online guided session or creating your own routine. Start 
with simple movements like neck rolls, shoulder stretches, and seated forward folds, followed by cat-cow stretches 
and downward-facing dog to release any tension. Focus on breathing deeply and mindfully throughout the process. 
This practice will invigorate your physical body while keeping you grounded in the peaceful ambiance of the day.',
    time_spent=0.25,
    from_state_id=None,
    to_state_id=None
)

...

StateVariables(
    context='You spent 30 to 45 minutes exploring a creative hobby as a continuation of your calm and intentional 
snowy Sunday. Whether drawing, knitting, or writing, this creative endeavor allowed you to express yourself freely 
in a cozy and serene atmosphere. The act of focusing on something deeply personal and engaging naturally uplifted 
your mood and provided a gentle mental challenge. This activity gave you an enriching sense of accomplishment while
adding another layer of meaning and individuality to your day. While your body remained relaxed from the lack of 
any strenuous activity, your mind and spirit benefited from this nourishing outlet for self-expression.',
    pain_level=23,
    pleasure_level=88,
    health_level=85,
    focus_level=80
)

Action(
    name='Watch a Comforting Movie or TV Show',
    details='Choose a comforting movie or a few episodes of a favorite TV show to watch in the later part of the 
afternoon. Set up a cozy space with blankets and a warm beverage like tea or hot cocoa. As you watch, allow 
yourself to simply relax and enjoy the entertainment without distraction. This activity serves as a soothing 
wind-down for the day while still keeping you engaged.',
    time_spent=1.5,
    from_state_id=None,
    to_state_id=None
)

StateVariables(
    context='After tidying the kitchen and maintaining the calm and intentional tone of your snowy Sunday, you 
transition to the later afternoon with a comforting activity: watching a movie or a favorite TV show. You create a 
cozy environment with blankets and prepare a warm beverage like tea or hot cocoa. Seated comfortably, you lean into
the relaxing mood of being entertained, appreciating the storytelling and visuals. This soothing activity not only 
reinforces the peaceful rhythm of your day but also allows your mind and body to fully unwind in preparation for 
the evening ahead.',
    pain_level=23,
    pleasure_level=90,
    health_level=85,
    focus_level=70
)

Action(
    name='Prepare for the Week Ahead',
    details='Later in the afternoon, reflect on the upcoming week and spend 30 minutes organizing and planning. 
Review any appointments, tasks, or commitments and jot them down in a planner or digital calendar. If needed, 
prepare any materials or make a simple to-do list. This action will help you transition smoothly from the restful 
Sunday into a productive start of the week, ensuring you feel poised and in control of your time.',
    time_spent=0.5,
    from_state_id=None,
    to_state_id=None
)

StateVariables(
    context='You took the opportunity in the late afternoon to spend 30 minutes organizing and planning for the 
week ahead. Sitting quietly with your planner or digital calendar, you reviewed appointments, tasks, and 
commitments, jotting them down methodically. The act of preparing a simple to-do list or assembling necessary 
materials helped you to mentally transition from the relaxing Sunday mindset into a proactive, prepared state. This
planning session reinforced a sense of calm control, easing any potential anxieties about the week to come while 
maintaining the tranquil energy of your day. The snowy day outside persisted, providing a serene and neutral 
backdrop to this reflective and practical activity.',
    pain_level=25,
    pleasure_level=80,
    health_level=83,
    focus_level=80
)

In [18]:
import pandas

pandas.options.plotting.backend = "plotly"

df = pandas.DataFrame(
    [
        {**x.state_variables.model_dump(), **x.environment_variables.model_dump()}
        for x in life.states
    ]
)[
    [
        "pain_level",
        "pleasure_level",
        "health_level",
        "focus_level",
        "hour_of_day",
    ]
]

In [19]:
df.set_index("hour_of_day").plot()